In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

import glob

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
# path = Path('../Resources/finished_data_dict.pkl')
# data_dict_import = load_obj(path)

In [5]:
## Import symbol list 
path = Path('../Resources/list_TI_export_success.pkl')
key_list = load_obj(path)
len(key_list)

932

In [6]:
# ## Load data 
# data_dict = {}

# for key in key_list:
#     path = Path('../FilesExport_DFs_with_TI_pkl/'+key+'_data_dict_with_technicals.pkl')
#     try:        
#         #df = pd.read_csv(file_path,index_col=0)
#         data = load_obj(path)
#         df = data[key]
#         data_dict[key] = df
#     except:
#         continue

In [7]:
# key_list = []
# for i in key_list_import_df['Symbol']:
#     key_list.append(i)
# len(key_list)

In [8]:
## Make a function to run this kind of model 
## Takes one symbol and runs model data. 
## Call function multiple times for multiple symbols 

def mean_squared_model(
    symbol,
    model_count = 9, ## Number of times model runs before saving the best one. Should be multiple of 3
    validation_split_value=0.3,
    epochs_value=800,
    units1 = 8,
    units2 = 8,
    units3 = 4,
    ):
    
    key = symbol 
    path = Path('../FilesExport_DFs_with_TI_pkl/'+key+'_data_dict_with_technicals.pkl')
      
    data = load_obj(path)
    import_df = data[key]
    df = import_df.copy()
    #data_dict[key] = df
        
    X = df.drop(columns={'close','adjClose'}).values
    y = df['close'].values
    scaler = StandardScaler().fit(X)
    X = scaler.transform(X)    
    
    num_of_inputs = 20
    num_of_outputs= 1
    layers = 1 ## Run the first model with one layer 

    for i in range(model_count):
        ## Create Neural Network 
        if layers == 1:
            # Create a shallow, 1 hidden layer, neural network
            nn = Sequential()

            # Hidden layer
            nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

            # Output layer
            nn.add(Dense(units=num_of_outputs, activation="linear"))

            # Compile the model
            nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

            # Fit the model
            model = nn.fit(X, y, 
                              validation_split=validation_split_value, 
                              epochs=epochs_value,
                              verbose=0)
            layers = 2 ## Run again with two layers 
            
        elif layers == 2:
            # Define the model - deep neural network with two layers
            nn = Sequential()

            # First hidden layer
            nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

            # Second hidden layer
            nn.add(Dense(units=units2, activation="relu"))

            # Output layer
            nn.add(Dense(units=num_of_outputs, activation="linear"))

            # Compile the model
            nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

            # Fit the model
            model = nn.fit(X, y, 
                              validation_split=validation_split_value, 
                              epochs=epochs_value, 
                              verbose=0)
            layers = 3 ## Run again with three layers 
            
        elif layers == 3:
            # Define the model - deep neural network with two layers
            nn = Sequential()

            # First hidden layer
            nn.add(Dense(units=units1, input_dim=num_of_inputs, activation="relu"))

            # Second hidden layer
            nn.add(Dense(units=units2, activation="relu"))
            
            # Third hidden layer
            nn.add(Dense(units=units3, activation="relu"))

            # Output layer
            nn.add(Dense(units=num_of_outputs, activation="linear"))

            # Compile the model
            nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

            # Fit the model
            model = nn.fit(X, y, 
                              validation_split=validation_split_value, 
                              epochs=epochs_value, 
                              verbose=0)
            layers = 1 ## Go back to 1 layer and re-run 
        ## End of if/elif

        model_loss, model_accuracy = nn.evaluate(X, y, verbose=0)
        
        ## Save first model 
        if i == 0:
            model_accuracy_high= model_accuracy
            model_accuracy_low = model_accuracy
            symbol_accuracy_dict_high[symbol] = model_accuracy_high
            symbol_accuracy_dict_low[symbol] = model_accuracy_low
            # Save model as JSON
            nn_json = nn.to_json()

            file_path = Path('../Model_Data_high_acc/'+symbol+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = ('../Model_Data_high_acc/'+symbol+'_model_weights.h5')
            nn.save_weights(file_path)
            
            file_path = Path('../Model_Data_low_acc/'+symbol+'_model_data.json')
            with open(file_path, "w") as json_file:
                json_file.write(nn_json)

            # Save weights
            file_path = ('../Model_Data_low_acc/'+symbol+'_model_weights.h5')
        ## Rewrite saved model if accuracy is higher or lower 
        else:
            if model_accuracy > model_accuracy_high:
                ## Rewrite values 
                model_accuracy_high = model_accuracy
                symbol_accuracy_dict_high[symbol] = model_accuracy_high
                
                ## Rewrite saved files 
                nn_json = nn.to_json()

                file_path = Path('../Model_Data_high_acc/'+symbol+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = ('../Model_Data_high_acc/'+symbol+'_model_weights.h5')
                nn.save_weights(file_path)
            elif model_accuracy < model_accuracy_low:
                ## Rewrite values 
                model_accuracy_low = model_accuracy
                symbol_accuracy_dict_low[symbol] = model_accuracy_low
                
                # Rewrite saved files 
                nn_json = nn.to_json()

                file_path = Path('../Model_Data_low_acc/'+symbol+'_model_data.json')
                with open(file_path, "w") as json_file:
                    json_file.write(nn_json)

                # Save weights
                file_path = ('../Model_Data_low_acc/'+symbol+'_model_weights.h5')
                nn.save_weights(file_path)
    ## End of for loop 
    ## Return nothing 

In [9]:
symbol_accuracy_dict_low = {}
symbol_accuracy_dict_high = {}
## Run test model 
mean_squared_model('GME',model_count=3)

In [10]:
## Code to load model and do predictions 

## Set Symbol 
symbol = 'GME'

# load json and create model
file_path = Path('../Model_Data_high_acc/'+symbol+'_model_data.json')
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = Path('../Model_Data_high_acc/'+symbol+'_model_weights.h5')
loaded_model.load_weights(file_path)

## Need to load data to apply to model 
key = symbol 
path = Path('../FilesExport_DFs_with_TI_pkl/'+key+'_data_dict_with_technicals.pkl')
data = load_obj(path)
import_df = data[key]
df = import_df.copy()

## Set X and y data and scale 
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values
scaler = StandardScaler().fit(X)
X = scaler.transform(X) 

## Copy dataframe and apply to model 
df2 = df.copy()
df2["predicted"] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]

df_pred 

,close,predicted
Date,,
2016-02-02,25.920000,25.853542
2016-02-03,26.170000,25.727924
2016-02-04,27.010000,26.741894
2016-02-05,26.830000,26.736399
2016-02-08,27.889999,27.769981
...,...,...
2021-10-25,173.970000,114.554626
2021-10-26,177.840000,116.913857
2021-10-27,173.510000,114.011879


In [11]:
# ##    Save accuracy values for each symbol, 
# ##    so we can find the best models to load afterwards
# symbol_accuracy_dict = {}